In [4]:
spark.stop()

In [2]:
import init

import findspark
findspark.init()

import os
import pandas as pd
import numpy as np
from pyspark.sql import *
from pyspark import *
import pyspark.sql.functions as f
from pyspark.sql.types import *
import time
from pyrecdp.data_processor import *
from pyrecdp.encoder import *
from pyrecdp.utils import *

spark = SparkSession.builder.master('yarn')\
        .appName("Recsys2021_data_process")\
        .getOrCreate()
path_prefix = "hdfs://"
dicts_folder = "recsys_dicts/"

In [ ]:
valid = spark.read.parquet("/recsys2021_0608_processed/sample_0_3_20days_eric_features/validate_with_features")
#train = spark.read.parquet("/recsys2021_0608_processed/sample_0_3_20days_eric_features/train_with_features")
train = spark.read.parquet("/recsys2021_0608_processed/sample_0_3_20days_eric_features/train_with_features_with_word_bucket_id")

In [ ]:
valid.printSchema()
train.printSchema()

In [ ]:
import pyspark.sql.functions as f

def tweet_word_to_max_bucketid(df, output):
    t1 = timer()
    tweet_dict_df = spark.read.parquet("/recsys2021_0608_processed/recsys_dicts/tweet_word_with_bucketid")
    df_with_id = df.withColumn('row_id', f.monotonically_increasing_id())
    df_with_id.write.format('parquet').mode('overwrite').save("file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/tmp_df_with_rowid")
    df_with_id = spark.read.parquet("file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/tmp_df_with_rowid")
    
    tmp_df = df_with_id\
               .select('row_id', 'tweet')\
               .withColumn('tweet_word', f.explode(f.split(f.col('tweet'), ' ')))\
               .join(tweet_dict_df.withColumnRenamed('dict_col', 'tweet_word').hint('shuffle_hash'), 'tweet_word', 'left')\
               .select('row_id', 'tweet_word', 'bucket_id')
    tmp_df.write.format('parquet').mode('overwrite').save("file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/tmp_valid_joined_tweet_word")
    tmp_df = spark.read.parquet("file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/tmp_valid_joined_tweet_word")
    most_used_wordbucketid_df = tmp_df\
           .groupby('row_id', 'bucket_id').agg(f.count(f.lit(1)).alias('wordcnt_by_bucketid'))\
           .withColumn('word_bucket_rank_id', f.row_number().over(Window.partitionBy('row_id').orderBy(f.desc('wordcnt_by_bucketid'))))\
           .where((f.col('word_bucket_rank_id') == 1) | (f.col('word_bucket_rank_id') == 2))\
           .select('row_id', 'bucket_id', 'wordcnt_by_bucketid', 'word_bucket_rank_id')
    most_used_wordbucketid_df.write.format('parquet').mode('overwrite').save("file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/tmp_most_uder_word_bucket_id_by_tweet")
    most_used_wordbucketid_df = spark.read.parquet("file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/tmp_most_uder_word_bucket_id_by_tweet")
    #most_used_wordbucketid_df.show()
    df = df_with_id.join(most_used_wordbucketid_df.filter("word_bucket_rank_id == 1"), 'row_id', 'left').drop('wordcnt_by_bucketid').drop('word_bucket_rank_id').withColumnRenamed('bucket_id', 'most_used_word_bucket_id')
    df = df.join(most_used_wordbucketid_df.filter("word_bucket_rank_id == 2"), 'row_id', 'left').drop('row_id').drop('wordcnt_by_bucketid').drop('word_bucket_rank_id').withColumnRenamed('bucket_id', 'second_used_word_bucket_id')
    df.write.format('parquet').mode('overwrite').save(output)
    
    t2 = timer()
    print("Feature Engineering for tweet text: encoded tweet column took %.3f" % (t2 - t1))
    return df

#train = spark.read.parquet("/recsys2021_0608_processed/sample_0_3_20days_eric_features/train_with_features")
valid = spark.read.parquet("/recsys2021_0608_processed/sample_0_3_20days_eric_features/validate_with_features")
tweet_word_to_max_bucketid(
    valid,
    "/recsys2021_0608_processed/sample_0_3_20days_eric_features/valid_with_features_with_word_bucket_id")

In [ ]:
from collections import *

path_prefix = "hdfs://"
dicts_folder = "recsys_dicts/"
current_path = "/recsys2021_0608_processed/sample_0_3_20days_eric_features/"
proc = DataProcessor(spark, path_prefix,
                     current_path=current_path, dicts_path=dicts_folder)

label_names = ['reply_timestamp', 'retweet_timestamp', 'retweet_with_comment_timestamp', 'like_timestamp']

te_dicts_train = OrderedDict() # value should be tuple, (input_cols, output_col, y_mean)
te_dicts_valid = OrderedDict() # value should be tuple, (input_cols, output_col, y_mean)
encoding_columns = []
train = spark.read.parquet("/recsys2021_0608_processed/sample_0_3_20days_eric_features/train_with_features_chendi")
#############################################################################################
### 1. Target Encoding ###
begin = time.time()
y_mean_all = []
y_mean_all_dict = {}
for t in ['reply_timestamp', 'retweet_timestamp', 'retweet_with_comment_timestamp', 'like_timestamp']:
    y_mean = np.array(train.groupBy().mean(t).collect())[0][0]
    y_mean_all.append(y_mean)
    y_mean_all_dict[t] = y_mean
    for c in ['most_used_word_bucket_id', 'second_used_word_bucket_id', 'mentioned_count', 'mentioned_bucket_id',
             ['has_mention', 'engaging_user_id'], ['mentioned_count', 'engaging_user_id'], ['mentioned_bucket_id', 'engaging_user_id']]:
        start = time.time()
        if isinstance(c, str):
            out_col = f'TE_{c}_{t}'
        elif isinstance(c, list):
            out_col = 'TE_'+'_'.join(c)+f'_{t}'
        encoder = TargetEncoder(proc, c, t, out_col, y_mean, out_dtype=FloatType())
        te_dicts_train[out_col], te_dicts_valid[out_col] = encoder.transform(train, valid, train_only=True)      
        print(out_col," y_mean is %f, %.1f seconds"%(y_mean, time.time()-start))
print(F"Target encoding #1 total time:{time.time()-begin}")
# #############################################################################################

In [131]:
lookup_table_path = "%s/train" % current_path
def merge_features_to_same_input(lookup_list):
    for c in lookup_list:
        to_join = []
        for t in ['reply_timestamp', 'retweet_timestamp', 'retweet_with_comment_timestamp', 'like_timestamp']:
            if isinstance(c, str):
                feature_name = f'TE_{c}_{t}'
                join_cols = ['fold', c]
            elif isinstance(c, list):
                feature_name = 'TE_'+'_'.join(c)+f'_{t}'
                join_cols = ['fold'] + c
            to_join.append(spark.read.parquet("%s/%s" % (lookup_table_path, feature_name)))
        df = to_join[0]
        for l in to_join[1:]:
            df = df.join(l, join_cols, 'left')
        output = "lookup_table_%s" % "_".join(join_cols)
        df.write.format('parquet').mode('overwrite').save("%s/train_lookup/%s" % (current_path, output))
    
def merge_features_to_main(train, lookup_list, output):
    for c in lookup_list:
        for t in ['reply_timestamp', 'retweet_timestamp', 'retweet_with_comment_timestamp', 'like_timestamp']:
            if isinstance(c, str):
                feature_name = f'TE_{c}_{t}'
                join_cols = ['fold', c]
            elif isinstance(c, list):
                feature_name = 'TE_'+'_'.join(c)+f'_{t}'
                join_cols = ['fold'] + c
        input_name = "lookup_table_%s" % "_".join(join_cols)
        lookup_df = spark.read.parquet("%s/train_lookup/%s" % (current_path, input_name))
        train = train.join(lookup_df, join_cols, 'left')
    train.write.format('parquet').mode('overwrite').save("%s/%s" % (current_path, output))

lookup_list_1 = ['most_used_word_bucket_id', 'second_used_word_bucket_id', 'mentioned_count', 'mentioned_bucket_id', ['has_mention', 'engaging_user_id']]
lookup_list_2 = [['mentioned_count', 'engaging_user_id'], ['mentioned_bucket_id', 'engaging_user_id']]
#merge_features_to_same_input(lookup_list_1)
#merge_features_to_same_input(lookup_list_2)
train = spark.read.parquet("/recsys2021_0608_processed/sample_0_3_20days_eric_features/train_with_features_chendi")
merge_features_to_main(train, lookup_list_1, "train_with_features_chendi_te_1")
train = spark.read.parquet("%s/%s" % (current_path, "train_with_features_chendi_te_1"))
merge_features_to_main(train, lookup_list_2, "train_with_features_chendi_te")

In [134]:
def gen_valid_lookup_from_train(lookup_list):
    for c in lookup_list:
        feature_list = []
        for t in ['reply_timestamp', 'retweet_timestamp', 'retweet_with_comment_timestamp', 'like_timestamp']:
            if isinstance(c, str):
                feature_name = f'TE_{c}_{t}'
                join_cols = ['fold', c]
                output = "lookup_table_%s" % c
            elif isinstance(c, list):
                feature_name = 'TE_'+'_'.join(c)+f'_{t}'
                join_cols = ['fold'] + c
                output = "lookup_table_%s" % "_".join(c)
            feature_list.append(feature_name)
        input_name = "lookup_table_%s" % "_".join(join_cols)
        #output = "lookup_table_%s" % c
        lookup_df = spark.read.parquet("%s/train_lookup/%s" % (current_path, input_name))
        valid_lookup_df = lookup_df.groupby(c).agg(*[f.mean(c).alias(c) for c in feature_list])
        valid_lookup_df.write.format('parquet').mode('overwrite').save("%s/valid_lookup/%s" % (current_path, output))

def merge_features_to_main(valid, lookup_list, output):
    for c in lookup_list:
        for t in ['reply_timestamp', 'retweet_timestamp', 'retweet_with_comment_timestamp', 'like_timestamp']:
            if isinstance(c, str):
                feature_name = f'TE_{c}_{t}'
                join_cols = [c]
            elif isinstance(c, list):
                feature_name = 'TE_'+'_'.join(c)+f'_{t}'
                join_cols = c
        input_name = "lookup_table_%s" % "_".join(join_cols)
        lookup_df = spark.read.parquet("%s/valid_lookup/%s" % (current_path, input_name))
        valid = valid.join(lookup_df, join_cols, 'left')
    valid.write.format('parquet').mode('overwrite').save("%s/%s" % (current_path, output))

lookup_list_1 = ['most_used_word_bucket_id', 'second_used_word_bucket_id', 'mentioned_count', 'mentioned_bucket_id', ['has_mention', 'engaging_user_id']]
lookup_list_2 = [['mentioned_count', 'engaging_user_id'], ['mentioned_bucket_id', 'engaging_user_id']]

valid = spark.read.parquet("%s/%s" % (current_path, "validate_with_features_chendi"))
gen_valid_lookup_from_train(lookup_list_1 + lookup_list_2)
merge_features_to_main(valid, lookup_list_1 + lookup_list_2, "validate_with_features_chendi_te_2")

In [130]:
#valid = spark.read.parquet("/recsys2021_0608_processed/sample_0_3_20days_eric_features/validate_with_features_chendi")
#valid.printSchema()
#valid.select('reply_timestamp', 'retweet_timestamp', 'retweet_with_comment_timestamp', 'like_timestamp',
#             'most_used_word_bucket_id', 'second_used_word_bucket_id', 'mentioned_count', 'mentioned_bucket_id').show(vertical=True)

train = spark.read.parquet("/recsys2021_0608_processed/sample_0_3_20days_eric_features/train_with_features_chendi_te")
#train.printSchema()
for c in train.columns:
    if "has_rt" not in c and ("has_" in c or "mention" in c or "word_bucket_id" in c):
        print(c)
train.select('reply_timestamp', 'retweet_timestamp', 'retweet_with_comment_timestamp', 'like_timestamp',
             'most_used_word_bucket_id', 'second_used_word_bucket_id', 'mentioned_count', 'mentioned_bucket_id').show(vertical=True)

mentioned_bucket_id
mentioned_count
has_mention
second_used_word_bucket_id
most_used_word_bucket_id
mention
has_photo
has_video
has_gif
has_links
has_hashtags
TE_most_used_word_bucket_id_reply_timestamp
TE_most_used_word_bucket_id_retweet_timestamp
TE_most_used_word_bucket_id_retweet_with_comment_timestamp
TE_most_used_word_bucket_id_like_timestamp
TE_second_used_word_bucket_id_reply_timestamp
TE_second_used_word_bucket_id_retweet_timestamp
TE_second_used_word_bucket_id_retweet_with_comment_timestamp
TE_second_used_word_bucket_id_like_timestamp
TE_mentioned_count_reply_timestamp
TE_mentioned_count_retweet_timestamp
TE_mentioned_count_retweet_with_comment_timestamp
TE_mentioned_count_like_timestamp
TE_mentioned_bucket_id_reply_timestamp
TE_mentioned_bucket_id_retweet_timestamp
TE_mentioned_bucket_id_retweet_with_comment_timestamp
TE_mentioned_bucket_id_like_timestamp
TE_has_mention_engaging_user_id_reply_timestamp
TE_has_mention_engaging_user_id_retweet_timestamp
TE_has_mention_engaging

In [2]:
train = spark.read.parquet("/recsys2021_0608_processed/sample_0_3_20days_eric_features/train_with_features_chendi_te")
train.drop("tweet_nortsign").sample(0.5, 3).write.format('parquet').mode('overwrite').save("/recsys2021_0608_processed/sample_0_3_20days_eric_features/train_with_features_chendi_0.15")

In [ ]:
import pyspark.sql.functions as F
word_dict_df = spark.read.parquet("/recsys2021_0608_processed/recsys_dicts/tweet_word_with_bucketid")
mention_dict_df = spark.read.parquet("/recsys2021_0608_processed/recsys_dicts/mention_with_bucketid")
train = spark.read.parquet("/recsys2021_0608_processed/sample_0_3_20days_eric_features/train_with_features_with_word_bucket_id")
train = train.withColumn('has_photo', F.col('present_media').contains('Photo').cast(IntegerType()))\
             .withColumn('has_video', F.col('present_media').contains('Vedio').cast(IntegerType()))\
             .withColumn('has_gif', F.col('present_media').contains('GIF').cast(IntegerType()))\
             .withColumn('has_links', (F.col('present_links') != "").cast(IntegerType()))\
             .withColumn('has_hashtags', (F.col('hashtags') != "").cast(IntegerType()))\
             .withColumn('mention', F.regexp_extract(F.col('tweet'), r"[^RT]\s@(\S+)", 1))\
             .withColumn('has_mention', (F.col('mention')!= "").cast(IntegerType()))\
             .withColumn('has_rt', (F.col('has_rt').cast(IntegerType())))
train = train.drop('text_tokens').drop('tokens').drop('tweet_nortsign')
train = train.join(mention_dict_df.hint('merge'), 'mention', 'left').withColumnRenamed('count', 'mentioned_count').withColumnRenamed('bucket_id', 'mentioned_bucket_id')
train.write.format('parquet').mode('overwrite').save("/recsys2021_0608_processed/sample_0_3_20days_eric_features/train_with_features_chendi")

In [ ]:
import pyspark.sql.functions as F
mention_dict_df = spark.read.parquet("/recsys2021_0608_processed/recsys_dicts/mention_with_bucketid")
valid = spark.read.parquet("/recsys2021_0608_processed/sample_0_3_20days_eric_features/valid_with_features_with_word_bucket_id")
valid = valid.withColumn('has_photo', F.col('present_media').contains('Photo').cast(IntegerType()))\
             .withColumn('has_video', F.col('present_media').contains('Vedio').cast(IntegerType()))\
             .withColumn('has_gif', F.col('present_media').contains('GIF').cast(IntegerType()))\
             .withColumn('has_links', (F.col('present_links') != "").cast(IntegerType()))\
             .withColumn('has_hashtags', (F.col('hashtags') != "").cast(IntegerType()))\
             .withColumn('mention', F.regexp_extract(F.col('tweet'), r"[^RT]\s@(\S+)", 1))\
             .withColumn('has_mention', (F.col('mention')!= "").cast(IntegerType()))\
             .withColumn('has_rt', (F.col('has_rt').cast(IntegerType())))
valid = valid.drop('text_tokens').drop('tokens').drop('tweet_nortsign')
valid = valid.join(mention_dict_df.hint('merge'), 'mention', 'left').withColumnRenamed('count', 'mentioned_count').withColumnRenamed('bucket_id', 'mentioned_bucket_id')
valid.write.format('parquet').mode('overwrite').save("/recsys2021_0608_processed/sample_0_3_20days_eric_features/validate_with_features_chendi")


In [ ]:
from pyspark.ml.feature import *
df = spark.read.parquet("/recsys2021_0608_processed/recsys_dicts/tweet")
df.printSchema()
qd = QuantileDiscretizer(numBuckets=100, inputCol="count", outputCol='bucket_id')
result  = qd.fit(df).transform(df)
#result.groupby('bucket_id').agg(F.min('count'), F.count(F.lit(1))).show()
result.write.format('parquet').mode('overwrite').save('/recsys2021_0608_processed/recsys_dicts/tweet_word_with_bucketid')

In [ ]:
df = spark.read.parquet("/recsys2021_0608_processed/recsys_dicts/tweet_word_with_bucketid")
df.printSchema()
df.groupby('bucket_id').agg(F.min('count'), F.count(F.lit(1))).show(50)
df = spark.read.parquet("/recsys2021_0608_processed/recsys_dicts/mention_with_bucketid")
df.printSchema()
df.groupby('bucket_id').agg(F.min('count'), F.count(F.lit(1))).show(50)
train = spark.read.parquet("/recsys2021_0608_processed/sample_0_3_20days_eric_features/train_with_features")
train.printSchema()